In [ ]:
from os.path import join, dirname, basename, splitext, abspath
from glob import glob
import pandas as pd
import yaml
import platform
from datetime import timedelta
from IPython.display import display
from IPython.core.display import display, HTML
import platform
from util.loader import load_raw_data, convert_data
display(HTML("<style>.container { width:100% !important; }</style>"))

this_folder = dirname(__file__) if '__file__' in globals() else abspath('')
root_folder = dirname(dirname(this_folder))
def get_test_result_folder(testname = 'baseline'):
    return join(root_folder, 'test', 'testresults', 'XPS-15-9560', testname, 'CUTEst')

def color_negative_red(val):
    color = 'red' if val < 0 else 'black'
    return f'color: {color}'
def color_negative_red_positive_green(val):
    if val > 0:
        return 'color: green'
    elif val < 0:
        return 'color: red'
    else:
        return ''

In [ ]:
tests = [
    'panoc-lbfgs-boxnqp',
    'panoc-slbfgs-boxnqp',
    'lbfgspp-boxnqp',
    'lbfgsbpp-boxnqp',
]
results_folders = [ get_test_result_folder(t) for t in tests ]

In [ ]:
results_raw = [ load_raw_data(f) for f in results_folders ]

In [ ]:
results_df = [ convert_data(r) for r in results_raw ]

In [ ]:
def df_stats(df):
    conv = df['status'].value_counts()['Converged']
    tot = df['status'].count()
    tot_time = df['time'].sum()
    conv_time = df[df['status'] == 'Converged']['time'].sum()
    print(f'Converged:      {conv}/{tot} = {100*conv/tot:.02f}%')
    print(f'Total time:     {tot_time:.03f}s')
    print(f'Converged time: {conv_time:.03f}s')

In [ ]:
for test, df in zip(tests, results_df):
    print(test + '\n---\n')
    df_stats(df)
    print('\n')

In [ ]:
all_converged = None 
for df in results_df:
      all_converged = (df['status'] == 'Converged') if all_converged is None else all_converged & (df['status'] == 'Converged')
print(f"Converged in all tests: {len(results_df[0][all_converged].index)}")

stats = [
    ('Time:                  ', lambda d: d['time'].sum()),
    ('Objective evaluations: ', lambda d: d['f evaluations'].sum()),
    ('Gradient evaluations:  ', lambda d: d['grad_f evaluations'].sum()),
    ('Outer iterations:      ', lambda d: d['outer iterations'].sum()),
    ('Inner iterations:      ', lambda d: d['inner iterations'].sum()),
    ('Linesearch failures:   ', lambda d: d['linesearch failures'].sum()),
    ('L-BFGS failures:       ', lambda d: d['L-BFGS failures'].sum()),
    ('L-BFGS rejected:       ', lambda d: d['L-BFGS rejected'].sum()),
]

print('\t\t\t' + '\t'.join(tests))
for stat, fun in stats:
      res = [ str(fun(df[all_converged])) for df in results_df]
      print(stat + '\t' + '\t'.join(res))

In [ ]:
df = results_df[0]
df[(df['status'] != 'Converged') & (df['n'] == 2)]